In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import broadcast

# Create Spark session
spark = SparkSession.builder \
    .appName("Join Types Lab") \
    .master("local[*]") \
    .getOrCreate()

# Sample Data
customers_data = [(1, "Marwa"), (2, "Medhat"), (3, "Ali"), (4, "Said")]
orders_data = [(101, 1, "iPhone"), (102, 1, "MacBook"), (103, 2, "T-Shirt"), (104, 5, "Shoes")]

# Create DataFrames
customers_df = spark.createDataFrame(customers_data, ["customer_id", "name"])
orders_df = spark.createDataFrame(orders_data, ["order_id", "customer_id", "product"])

# Register as temp views for SQL
customers_df.createOrReplaceTempView("customers")
orders_df.createOrReplaceTempView("orders")



# Inner Join
customers_df.join(orders_df, "customer_id", "inner").show()

# SQL
spark.sql("""
SELECT c.*, o.order_id, o.product 
FROM customers c 
JOIN orders o ON c.customer_id = o.customer_id
""").show()


# Left Join
customers_df.join(orders_df, "customer_id", "left").show()


# Right Join
customers_df.join(orders_df, "customer_id", "right").show()


# Full Outer Join
customers_df.join(orders_df, "customer_id", "outer").show()


# Broadcast Join (Optimization)
# Broadcast is useful when one table is small enough to fit in memory and avoid shuffling large data.
# Broadcasting small customers_df
orders_df.join(broadcast(customers_df), "customer_id").show()

+-----------+------+--------+-------+
|customer_id|  name|order_id|product|
+-----------+------+--------+-------+
|          1| Marwa|     101| iPhone|
|          1| Marwa|     102|MacBook|
|          2|Medhat|     103|T-Shirt|
+-----------+------+--------+-------+

+-----------+------+--------+-------+
|customer_id|  name|order_id|product|
+-----------+------+--------+-------+
|          1| Marwa|     101| iPhone|
|          1| Marwa|     102|MacBook|
|          2|Medhat|     103|T-Shirt|
+-----------+------+--------+-------+

+-----------+------+--------+-------+
|customer_id|  name|order_id|product|
+-----------+------+--------+-------+
|          1| Marwa|     102|MacBook|
|          1| Marwa|     101| iPhone|
|          2|Medhat|     103|T-Shirt|
|          3|   Ali|    null|   null|
|          4|  Said|    null|   null|
+-----------+------+--------+-------+

+-----------+------+--------+-------+
|customer_id|  name|order_id|product|
+-----------+------+--------+-------+
|        